In [1]:
#load python packages
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score

In [2]:
#This is a hack to cut down on the pandas/sklearn warnings.
#My notebook runs fine so I wanted to make it more readable.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
def get_file(file_name):
    path = os.getcwd()
    path += "/../data/processed/"+file_name
    #have to eval text columns (which are currently lists of strings) or pandas will read a list as one big string instead
    df = pd.read_csv(path, index_col='job_id',\
                     converters={'company_profile': eval,'description': eval, 'requirements':eval})
    return df

In [4]:
X_train = get_file('fake_job_postings_X_train.csv')
y_train = get_file('fake_job_postings_y_train.csv')
X_test = get_file('fake_job_postings_X_test.csv')
y_test = get_file('fake_job_postings_y_test.csv')

Model: Naive Bayes

In [5]:
def MultinomialNB_predict(df_col,y):
    df_col_together = df_col.str.join(sep=' ')
    cv = CountVectorizer()
    df_col_tf = cv.fit_transform(df_col_together)
    mnb = MultinomialNB()
    return cross_val_score(mnb,df_col_tf,y, n_jobs=-1, scoring='f1').mean()

In [6]:
word_features = ['company_profile', 'description', 'requirements'] 
for feature in word_features:
    print('feature: '+feature+'\nF1: ' +str(MultinomialNB_predict(X_train[feature],y_train))+'\n')

feature: company_profile
F1: 0.3661781462868491

feature: description
F1: 0.36058733496247414

feature: requirements
F1: 0.21513278384806314



In [7]:
combined_text_cols = X_train['company_profile']+ X_train['description']+ X_train['requirements']
print('feature: '+'all text combined'+'\nF1: ' +str(MultinomialNB_predict(combined_text_cols,y_train))+'\n')

feature: all text combined
F1: 0.45366096507074855



Model: Logistic Regression

In [8]:
X_train_no_words = X_train.drop(word_features, axis=1)
logreg_model = LogisticRegression()
cross_val_score(logreg_model,X_train_no_words,y_train, n_jobs=-1, scoring='f1').mean()

0.34650865035930445

Model: Random Forest

In [9]:
randforest_model = RandomForestClassifier()
cross_val_score(randforest_model,X_train_no_words,y_train, n_jobs=-1, scoring='f1').mean()

/home/joe/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/joe/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/joe/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/joe/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: 

0.6967566506144455

Conclusion: Best model and performance on hold-out data

Our best model so far (measured by F1 score) has been the out of the box RandomForestClassifier from sklearn, using the non-word data. I'd like to end the modelling section by training this model on all of our training data and then scoring based on the test data.

In [10]:
final_model = RandomForestClassifier()
final_model.fit(X_train_no_words, y_train)
X_test_no_words = X_test.drop(word_features, axis=1)
y_pred = final_model.predict(X_test_no_words)
print(metrics.classification_report(y_test, y_pred))

/home/joe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3777
           1       0.98      0.60      0.74       196

    accuracy                           0.98      3973
   macro avg       0.98      0.80      0.87      3973
weighted avg       0.98      0.98      0.98      3973



In [11]:
print(metrics.confusion_matrix(y_test,y_pred))

[[3774    3]
 [  78  118]]


The performance is about as good as I was hoping for!